In [1]:
import os

In [2]:
%pwd

'/media/kirti/Dev/DeepLearning/Project/E2E/ChestCancerDetection/research'

In [3]:
os.chdir('../')
%pwd    

'/media/kirti/Dev/DeepLearning/Project/E2E/ChestCancerDetection'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        print(self.config.artifacts_root)
        create_directories([self.config.artifacts_root]) 
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [20]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_file_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        '''
        Downloads data from the source URL to the local data file path.
        If the file already exists, it skips the download.
        '''

        try:
            dataset_url = self.config.source_url
            zip_download_path = self.config.local_data_file
            unzip_dir = self.config.unzip_dir
            os.makedirs(os.path.dirname(unzip_dir), exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_path}")

            file_id = dataset_url.split('/')[-2]
            prefix = f"https://drive.google.com/uc?export=download&id={file_id}"
            if os.path.exists(zip_download_path):
                file_size = get_file_size(Path(zip_download_path))
                if file_size > 0:
                    logger.info(f"File already exists at {zip_download_path} with size {file_size} bytes. Skipping download.")
                    return
            gdown.download(prefix, zip_download_path, quiet=False)

            logger.info(f"Downloaded data to {zip_download_path}")

        except Exception as e:
            logger.error(f"Error downloading data: {e}")
            raise e  

    def unzip_data(self):
        '''
        Unzips the downloaded data file to the specified unzip directory.
        If the unzip directory already exists, it skips the unzipping.
        '''

        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Unzipped data to {self.config.unzip_dir}")
        

In [28]:
try:
    config_manager = ConfigManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_data()
    data_ingestion.unzip_data() 

except Exception as e:
    logger.error(f"Error in data ingestion pipeline: {e}")
    raise e

artifacts
[2025-07-06 17:14:23,228|(INFO)| File: common | Message: Created directory: artifacts]
[2025-07-06 17:14:23,235|(INFO)| File: common | Message: Created directory: artifacts/data_ingestion]
[2025-07-06 17:14:23,239|(INFO)| File: 2734619864 | Message: Downloading data from https://drive.google.com/file/d/1ArZQhYChicOaM2AFE7dj0FNDzQPwBW7L/view?usp=sharing to artifacts/data_ingestion/data.zip]
[2025-07-06 17:14:23,245|(INFO)| File: 2734619864 | Message: File already exists at artifacts/data_ingestion/data.zip with size 124379012 bytes. Skipping download.]
[2025-07-06 17:14:31,278|(INFO)| File: 2734619864 | Message: Unzipped data to artifacts/data_ingestion]
